In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install pytorch

In [ ]:
import torch
import torchvision.models as models
import torchvision
from collections import Counter
from distutils.dir_util import copy_tree
import matplotlib.pyplot as plt
from pathlib import Path

from torchvision import transforms,datasets
import cv2

In [ ]:
model = models.resnet18(pretrained=False)
model.fc=torch.nn.Linear(512,5)

fromDirectory = "../input/aptos2019-blindness-detection/train_images/"
toDirectory = "./train_images/"
#without output
_=copy_tree(fromDirectory, toDirectory)

In [ ]:
transformed_dir="./train_images/"
Path(transformed_dir).mkdir(parents=True, exist_ok=True)
train_dir="../input/aptos2019-blindness-detection/train_images/"
test_dir="../input/aptos2019-blindness-detection/test_images/"
df=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
#loss = torch.nn.CrossEntropyLoss()
loss=torch.nn.MSELoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)

In [ ]:
Counter(list(df.diagnosis))

In [ ]:
diagnosis_count=Counter(list(df.diagnosis))
max_diagnosis_count=max(diagnosis_count.values())
print(max_diagnosis_count)
#what about int(round(...))?
transforms_count={i:max_diagnosis_count//diagnosis_count[i]-1 for i in diagnosis_count.keys()}

In [ ]:
transforms_count

In [ ]:
def return_n_transformed_images(orig_img,n:int):
    train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomAutocontrast(),
    transforms.ColorJitter(brightness=(0.7,1.0), contrast=(0.7,1.0)),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))])
])
    return [train_transforms(orig_img) for i in range(n)]

In [ ]:
def return_n_transformed_images_stub(orig_img,n:int):
    train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.0),
])
    return [train_transforms(orig_img) for i in range(n)]

image_path=os.path.join("../input/aptos2019-blindness-detection/train_images/000c1434d8d7.png")
img = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
img=transforms.ToTensor()(img)
tr_imgs=return_n_transformed_images(img,1)
transforms.ToPILImage()(tr_imgs[0])

In [ ]:
df

In [ ]:
#fill df
c=len(df)-1
outs=[]
for i in df.values:
    image_id,diagnosis=i
    image_path=os.path.join(train_dir,image_id+".png")
    img = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
    img=transforms.ToTensor()(img)
    img.to(device)
    for transformed_image in return_n_transformed_images_stub(img,transforms_count[diagnosis]):
        transformed_image.to(device)
        c+=1
        pil = transforms.ToPILImage()(transformed_image)
        file_path=os.path.join(transformed_dir,str(c)+".png")
        if c%10==0:
            print(c,end=" ")
        pil.save(file_path)
        outs.append([file_path,diagnosis])

In [ ]:
aug_only_df=pd.DataFrame(outs,columns=["id_code","diagnosis"],index=list(map(lambda x:x[0],outs)))

In [ ]:
image_path

In [ ]:
aug_only_df=pd.DataFrame(outs,columns=["id_code","diagnosis"],index=list(map(lambda x:x[0],outs)))

In [ ]:
df.id_code=train_dir+df.id_code+".png"

In [ ]:
df

In [ ]:
df=df.append(aug_only_df)

In [ ]:
df=df.sample(frac=1)

In [ ]:
df

In [ ]:
df.diagnosis.hist(bins=5)

In [ ]:
c=0
for i in df.values:
    image_path,diagnosis=i
    img = cv2.imread(image_path)
    img=transforms.ToTensor()(img).unsqueeze_(0)/256
    img=img.to(device)
    img=return_n_transformed_images(img,1)[0]
    true_tensor=[0.0,0.0,0.0,0.0,0.0]
    true_tensor[diagnosis]=1.0
    true_tensor=torch.Tensor(true_tensor).to(device)
    optimizer.zero_grad()
    preds=model.forward(img)
    loss_value = loss(preds[0], true_tensor)
    loss_value.backward()
    optimizer.step()
    c+=1
    if c%20==0:
        print(c, loss_value)

In [ ]:
df=pd.read_csv("../input/aptos2019-blindness-detection/test.csv")

In [ ]:
df

In [ ]:
out=[]
c=0
for i in df.values:
    image_id=i[0]
    image_path=os.path.join(test_dir,image_id+".png")
    img = cv2.imread(image_path)
    img=transforms.ToTensor()(img).unsqueeze_(0)/256
    img=img.to(device)
    preds=model.forward(img)
    max_class=max(preds[0].tolist())
    max_index=preds[0].tolist().index(max_class)
    out.append([image_id,max_index])
    c+=1
    if c%50==0:
        print(c)

In [ ]:
df = pd.DataFrame(out)
df.columns=["id_code","diagnosis"]
df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
df.diagnosis.hist(bins=5)